# 📚 PCA + ANN Face Recognition Project

In [ ]:
# 1. Setup: Install & Import Libraries
!pip install opencv-python

import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# 2. Load Dataset
dataset_path = '/path/to/dataset/faces/'  # <-- CHANGE THIS

images = []
labels = []
label_dict = {}

for i, person in enumerate(os.listdir(dataset_path)):
    person_path = os.path.join(dataset_path, person)
    if os.path.isdir(person_path):
        label_dict[i] = person
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (100, 100))
            images.append(img.flatten())
            labels.append(i)

images = np.array(images).T
labels = np.array(labels)

print(f"Image matrix shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

In [ ]:
# 3. Mean Calculation
mean_face = np.mean(images, axis=1).reshape(-1,1)
plt.imshow(mean_face.reshape(100, 100), cmap='gray')
plt.title("Mean Face")
plt.show()

In [ ]:
# 4. Mean Zero (Delta)
delta = images - mean_face

In [ ]:
# 5. Covariance Matrix (Surrogate Covariance)
covariance_matrix = np.dot(delta.T, delta)
print(f"Covariance matrix shape: {covariance_matrix.shape}")

In [ ]:
# 6. Eigen Decomposition
eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

idx = np.argsort(-eigenvalues)
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

eigenfaces = np.dot(delta, eigenvectors)
eigenfaces = eigenfaces / np.linalg.norm(eigenfaces, axis=0)

In [ ]:
# 7. Select Top k Eigenfaces (Feature Vectors)
k = 50  # Example value

selected_eigenfaces = eigenfaces[:, :k]
print(f"Selected eigenfaces shape: {selected_eigenfaces.shape}")

for i in range(5):
    plt.imshow(selected_eigenfaces[:,i].reshape(100,100), cmap='gray')
    plt.title(f"Eigenface {i+1}")
    plt.show()

In [ ]:
# 8. Generate Signature for Each Face (Omega)
omega = np.dot(selected_eigenfaces.T, delta)
print(f"Signature matrix shape: {omega.shape}")

In [ ]:
# 9. Train ANN Classifier
X_train, X_test, y_train, y_test = train_test_split(omega.T, labels, test_size=0.4, random_state=42)

ann = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
ann.fit(X_train, y_train)

y_pred = ann.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc*100:.2f}%")

In [ ]:
# 10. Testing with a New Image
test_img_path = '/path/to/test_image.jpg'  # <-- CHANGE THIS
test_img = cv2.imread(test_img_path, cv2.IMREAD_GRAYSCALE)
test_img = cv2.resize(test_img, (100,100))
test_img_flat = test_img.flatten().reshape(-1,1)

test_img_delta = test_img_flat - mean_face
omega_test = np.dot(selected_eigenfaces.T, test_img_delta).T

predicted_label = ann.predict(omega_test)
print(f"Predicted Label: {label_dict[predicted_label[0]]}")

In [ ]:
# 11. Plot Accuracy vs k Value
k_values = range(10, min(images.shape[1], 150), 10)
accuracies = []

for k in k_values:
    selected_eigenfaces = eigenfaces[:, :k]
    omega = np.dot(selected_eigenfaces.T, delta)

    X_train, X_test, y_train, y_test = train_test_split(omega.T, labels, test_size=0.4, random_state=42)
    ann = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
    ann.fit(X_train, y_train)
    y_pred = ann.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))

plt.plot(k_values, accuracies, marker='o')
plt.xlabel("k (Number of Eigenfaces)")
plt.ylabel("Accuracy")
plt.title("Accuracy vs k Value")
plt.grid()
plt.show()

In [ ]:
# 12. Imposter Detection (Bonus)
imposter_img_path = '/path/to/imposter_image.jpg'  # <-- CHANGE THIS
imposter_img = cv2.imread(imposter_img_path, cv2.IMREAD_GRAYSCALE)
imposter_img = cv2.resize(imposter_img, (100,100))
imposter_img_flat = imposter_img.flatten().reshape(-1,1)

imposter_delta = imposter_img_flat - mean_face
omega_imposter = np.dot(selected_eigenfaces.T, imposter_delta).T

imposter_label = ann.predict(omega_imposter)
print(f"Predicted Label (Imposter): {label_dict.get(imposter_label[0], 'Unknown')}")